In [1]:
from sparql_llm import SparqlExamplesLoader, SparqlInfoLoader, SparqlVoidShapesLoader
from langchain_core.documents import Document


In [2]:
endpoints = [
    "https://sparql.uniprot.org/sparql/",
    "https://sparql.rhea-db.org/sparql/",
    "https://beta.sparql.swisslipids.org/",
    "https://biosoda.unil.ch/emi/sparql/"
]

In [3]:
from typing import List, Dict, Optional
from qdrant_client import QdrantClient
from qdrant_client.http.models import SearchRequest, NamedVector, NamedSparseVector, SparseIndexParams, SparseVector
from langchain_qdrant.fastembed_sparse import FastEmbedSparse
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
import gc
import time
from langchain_qdrant import QdrantVectorStore



def init_endpoint_examples(endpoints: List[str], collection_name: str = "biomedical_examples_collection_v1.0", embedding_model: str = "BAAI/bge-base-en-v1.5", parallel: int = 4) -> None:

    docs = []
 
    
    for endpoint in endpoints:
        docs += SparqlExamplesLoader(endpoint_url=endpoint).load()
    

    print(f"Loaded {len(docs)} documents from {len(endpoints)} endpoints")

    start_time = time.time()

    QdrantVectorStore.from_documents(
        docs,
        host="localhost",
        grpc_port=6334,
        prefer_grpc=True,
        collection_name=collection_name,
        force_recreate=True,
        embedding= FastEmbedEmbeddings(model_name=embedding_model, parallel=parallel),
    )

    print(f"Done generating and indexing {len(docs)} documents into the vectordb in {time.time() - start_time} seconds")



    

In [4]:
init_endpoint_examples(endpoints)
    

Found 111 examples queries for https://sparql.uniprot.org/sparql/
Found 121 examples queries for https://sparql.rhea-db.org/sparql/
Found 23 examples queries for https://beta.sparql.swisslipids.org/
Found 19 examples queries for https://biosoda.unil.ch/emi/sparql/


Loaded 274 documents from 4 endpoints
Done generating and indexing 274 documents into the vectordb in 39.81329107284546 seconds


In [5]:
import os
import sys

sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

In [6]:
print(docs[0])

NameError: name 'docs' is not defined

In [ ]:

client = QdrantClient(
            host="localhost",
            grpc_port=6334,
            prefer_grpc=True,
            timeout=60
        )


for question i

In [ ]:
embeddings = FastEmbedEmbeddings(
            model_name=rag_config.dense_embedding_model,
            cache_dir=cache_dir,
        )
        
        # Initialize the Qdrant client
        logger.info(f"Connecting to Qdrant at {vectordb_url}")
        client = QdrantClient(
            host=rag_config.host,
            grpc_port=rag_config.grpc_port,
            prefer_grpc=True,
            timeout=rag_config.timeout
        )
        
        entities_list = []
        classes_list = []
        steps = []
        
        # Get all entities to process
        entities = state.structured_question.extracted_entities
        
        # Get all classes to process
        classes = state.structured_question.extracted_classes
        
        has_entities_or_classes = False
        
        # Process entities if they exist
        if entities:
            has_entities_or_classes = True
            # Generate embeddings for all entities in a single batch
            logger.info(f"Generating batch embeddings for {len(entities)} entities")
            entity_embeddings = embeddings.embed_documents(entities)
            
            # Process each entity with its embedding
            for i, potential_entity in enumerate(entities):
                try:
                    dense_vector = entity_embeddings[i]
                    
                    if dense_vector is not None:
                        # Use direct query_points method as requested
                        logger.info(f"Querying Qdrant for entity: {potential_entity}")
                        results = client.query_points(
                            collection_name=collection_name,
                            query=dense_vector,
                            using="dense",
                            with_payload=True,
                            limit=top_k,
                        )